# Problem Statement

### Credit Card Lead Prediction

Happy Customer Bank is a mid-sized private bank that deals in all kinds of banking products, like Savings accounts, Current accounts, investment products, credit products, among other offerings.


The bank also cross-sells products to its existing customers and to do so they use different kinds of communication like tele-calling, e-mails, recommendations on net banking, mobile banking, etc. 


In this case, the Happy Customer Bank wants to cross sell its credit cards to its existing customers. The bank has identified a set of customers that are eligible for taking these credit cards.


Now, the bank is looking for your help in identifying customers that could show higher intent towards a recommended credit card, given:

    Customer details (gender, age, region etc.)
    Details of his/her relationship with the bank (Channel_Code,Vintage, 'Avg_Asset_Value etc.)


In [1]:
# !wget https://datahack-prod.s3.amazonaws.com/train_file/train_s3TEQDk.csv
# !wget https://datahack-prod.s3.amazonaws.com/test_file/test_mSzZ8RL.csv
# pip install -U scikit-learn

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelEncoder
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score

from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

pd.options.display.max_columns = 300

In [3]:
data = pd.read_csv("train_s3TEQDk.csv")
test = pd.read_csv("test_mSzZ8RL.csv")
print(f"Train shape {data.shape}, Test Shape {test.shape}")

Train shape (245725, 11), Test Shape (105312, 10)


In [4]:
train,valid = train_test_split(data,test_size=0.20,random_state=345,stratify=data['Is_Lead'])
train = train.copy()
valid = valid.copy()
print(f"Train shape {train.shape} Validation shape {valid.shape}")

Train shape (196580, 11) Validation shape (49145, 11)


In [5]:
train.head(3)

,ID,Gender,Age,Region_Code,Occupation,Channel_Code,Vintage,Credit_Product,Avg_Account_Balance,Is_Active,Is_Lead
53078,N3ZQ84QR,Female,46,RG280,Self_Employed,X2,51,No,863584,Yes,0
213644,JWGAMK7P,Male,67,RG258,Other,X2,43,Yes,706126,No,0
131870,CX9NGNQT,Male,46,RG279,Self_Employed,X2,26,Yes,422207,Yes,0


In [6]:
test.head(3)

,ID,Gender,Age,Region_Code,Occupation,Channel_Code,Vintage,Credit_Product,Avg_Account_Balance,Is_Active
0,VBENBARO,Male,29,RG254,Other,X1,25,Yes,742366,No
1,CCMEWNKY,Male,43,RG268,Other,X2,49,NaN,925537,No
2,VK3KGA9M,Male,31,RG270,Salaried,X1,14,No,215949,No


In [7]:
valid.head(3)

,ID,Gender,Age,Region_Code,Occupation,Channel_Code,Vintage,Credit_Product,Avg_Account_Balance,Is_Active,Is_Lead
148453,OK9KJGZ2,Female,55,RG268,Self_Employed,X1,37,No,929257,No,0
117997,TTC7CPSI,Male,57,RG283,Self_Employed,X3,87,NaN,909740,No,0
5432,MPUWVRAX,Male,39,RG275,Salaried,X1,8,Yes,961742,Yes,0


In [8]:
train['ID'].nunique()

196580

In [9]:
train.isna().sum()

ID                         0
Gender                     0
Age                        0
Region_Code                0
Occupation                 0
Channel_Code               0
Vintage                    0
Credit_Product         23525
Avg_Account_Balance        0
Is_Active                  0
Is_Lead                    0
dtype: int64

In [10]:
train['Gender'].value_counts(normalize=True)

Male      0.546693
Female    0.453307
Name: Gender, dtype: float64

In [11]:
train['Region_Code'].nunique()

35

In [12]:
train['Occupation'].value_counts(normalize=True)

Self_Employed    0.411532
Salaried         0.292731
Other            0.284866
Entrepreneur     0.010871
Name: Occupation, dtype: float64

In [13]:
train['Channel_Code'].value_counts(normalize=True)

X1    0.421279
X3    0.280359
X2    0.275669
X4    0.022693
Name: Channel_Code, dtype: float64

In [14]:
train['Credit_Product'].value_counts(normalize=True)

No     0.667019
Yes    0.332981
Name: Credit_Product, dtype: float64

In [15]:
train['Avg_Account_Balance'].describe()

count    1.965800e+05
mean     1.129489e+06
std      8.532486e+05
min      2.079000e+04
25%      6.042470e+05
50%      8.954865e+05
75%      1.368733e+06
max      1.035201e+07
Name: Avg_Account_Balance, dtype: float64

In [16]:
train['Is_Active'].value_counts(normalize=True)

No     0.611375
Yes    0.388625
Name: Is_Active, dtype: float64

In [17]:
train['Is_Lead'].value_counts(normalize=True)

0    0.762794
1    0.237206
Name: Is_Lead, dtype: float64

In [18]:
train['Age'].describe()

count    196580.000000
mean         43.864971
std          14.821238
min          23.000000
25%          30.000000
50%          43.000000
75%          54.000000
max          85.000000
Name: Age, dtype: float64

In [19]:
train['Vintage'].describe()

count    196580.000000
mean         46.978121
std          32.346981
min           7.000000
25%          20.000000
50%          32.000000
75%          73.000000
max         135.000000
Name: Vintage, dtype: float64

In [20]:
train.groupby(['Is_Lead'])[['Age','Avg_Account_Balance']].median()

,Age,Avg_Account_Balance
Is_Lead,,
0,38,871158
1,49,980686


In [21]:
train['Credit_Product'] = train['Credit_Product'].fillna('NA')
train['Avg_Account_Balance'] = np.log(1+train['Avg_Account_Balance'])

test['Credit_Product'] = test['Credit_Product'].fillna('NA')
test['Avg_Account_Balance'] = np.log(1+test['Avg_Account_Balance'])

valid['Credit_Product'] = valid['Credit_Product'].fillna('NA')
valid['Avg_Account_Balance'] = np.log(1+valid['Avg_Account_Balance'])


In [22]:
train.sample(3)

,ID,Gender,Age,Region_Code,Occupation,Channel_Code,Vintage,Credit_Product,Avg_Account_Balance,Is_Active,Is_Lead
31990,PKVKG43V,Male,33,RG283,Self_Employed,X2,37,Yes,13.621458,Yes,1
33526,YYYAHISF,Male,56,RG269,Other,X4,15,No,13.661380,Yes,0
74313,HDXFTDTW,Male,38,RG261,Salaried,X3,19,Yes,13.920527,No,0


In [23]:
train['Is_Active'] = train['Is_Active'].replace({'No':'N','Yes':'Y'})
test['Is_Active'] = test['Is_Active'].replace({'No':'N','Yes':'Y'})
valid['Is_Active'] = valid['Is_Active'].replace({'No':'N','Yes':'Y'})

In [24]:
def bucket_age(age):
    if age <= 25:
        val = '0-25'
    elif age > 25 and age <=30:
        val = '25-30'
    elif age > 30 and age <= 35:
        val = '30-35'
    elif age > 35 and age <=40:
        val = '35-40'
    elif age > 40 and age <=45:
        val = '40-45'
    elif age > 45 and age <=50:
        val = '45-50'
    elif age > 50 and age <= 55:
        val = '50-55'
    elif age > 55 and age <= 60:
        val = '55-60'
    elif age > 60 and age <=65:
        val = '60-65'
    elif age >65 and age <=70:
        val = '65-70'
    else:
        val = '70+'
    return val

train['age_cat'] = train['Age'].apply(bucket_age)
test['age_cat'] = test['Age'].apply(bucket_age)
valid['age_cat'] = valid['Age'].apply(bucket_age)
train[['Age','age_cat']].sample(3)

,Age,age_cat
108575,41,40-45
238977,48,45-50
142863,29,25-30


In [27]:
train[['Age','age_cat']].sample(3)

,Age,age_cat
74574,58,55-60
99151,48,45-50
180014,43,40-45


In [28]:
cat_cols = ['age_cat','Gender','Region_Code','Occupation','Channel_Code','Credit_Product','Is_Active']
featured_cols = []
for idx,col in enumerate(cat_cols):
    for sub_col in cat_cols[idx+1:]:
        new_col = f"{col}-{sub_col}"
        featured_cols.append(new_col)
        train[new_col] = train[col] + "-" + train[sub_col]
        test[new_col] = test[col] + "-" + test[sub_col]
        valid[new_col] = valid[col] + "-" + valid[sub_col]

train.sample(3)

,ID,Gender,Age,Region_Code,Occupation,Channel_Code,Vintage,Credit_Product,Avg_Account_Balance,Is_Active,Is_Lead,age_cat,age_cat-Gender,age_cat-Region_Code,age_cat-Occupation,age_cat-Channel_Code,age_cat-Credit_Product,age_cat-Is_Active,Gender-Region_Code,Gender-Occupation,Gender-Channel_Code,Gender-Credit_Product,Gender-Is_Active,Region_Code-Occupation,Region_Code-Channel_Code,Region_Code-Credit_Product,Region_Code-Is_Active,Occupation-Channel_Code,Occupation-Credit_Product,Occupation-Is_Active,Channel_Code-Credit_Product,Channel_Code-Is_Active,Credit_Product-Is_Active
220460,3DZJLMZW,Female,51,RG254,Self_Employed,X2,86,No,13.666164,Y,0,50-55,50-55-Female,50-55-RG254,50-55-Self_Employed,50-55-X2,50-55-No,50-55-Y,Female-RG254,Female-Self_Employed,Female-X2,Female-No,Female-Y,RG254-Self_Employed,RG254-X2,RG254-No,RG254-Y,Self_Employed-X2,Self_Employed-No,Self_Employed-Y,X2-No,X2-Y,No-Y
191766,5NMCCFZ4,Female,69,RG258,Other,X2,74,Yes,13.677335,N,1,65-70,65-70-Female,65-70-RG258,65-70-Other,65-70-X2,65-70-Yes,65-70-N,Female-RG258,Female-Other,Female-X2,Female-Yes,Female-N,RG258-Other,RG258-X2,RG258-Yes,RG258-N,Other-X2,Other-Yes,Other-N,X2-Yes,X2-N,Yes-N
99947,EOT4S7CG,Female,32,RG272,Salaried,X1,26,NA,13.650617,N,1,30-35,30-35-Female,30-35-RG272,30-35-Salaried,30-35-X1,30-35-NA,30-35-N,Female-RG272,Female-Salaried,Female-X1,Female-NA,Female-N,RG272-Salaried,RG272-X1,RG272-NA,RG272-N,Salaried-X1,Salaried-NA,Salaried-N,X1-NA,X1-N,NA-N


In [29]:
all_cat_cols = cat_cols + featured_cols
num_col = ['Vintage','Avg_Account_Balance']
for idx,col in enumerate (all_cat_cols):
    print(f"\rWorking Cat Col {idx}/{len(all_cat_cols)}",end='')
    for ind, num in enumerate(num_col):

        grp = train.groupby([col])[num].agg(['mean','std'])
        grp['cv'] = grp['std']/(1+grp['mean'])
        grp = grp.add_prefix(f'{col}-{num}-')
        grp = grp.fillna(-1)
        grp = grp.reset_index()
        
        train = train.merge(grp,on=[col],how='left')
        test = test.merge(grp,on=[col],how='left')
        valid = valid.merge(grp,on=[col],how='left')

Working Cat Col 27/28

In [30]:
train.head(3)

,ID,Gender,Age,Region_Code,Occupation,Channel_Code,Vintage,Credit_Product,Avg_Account_Balance,Is_Active,Is_Lead,age_cat,age_cat-Gender,age_cat-Region_Code,age_cat-Occupation,age_cat-Channel_Code,age_cat-Credit_Product,age_cat-Is_Active,Gender-Region_Code,Gender-Occupation,Gender-Channel_Code,Gender-Credit_Product,Gender-Is_Active,Region_Code-Occupation,Region_Code-Channel_Code,Region_Code-Credit_Product,Region_Code-Is_Active,Occupation-Channel_Code,Occupation-Credit_Product,Occupation-Is_Active,Channel_Code-Credit_Product,Channel_Code-Is_Active,Credit_Product-Is_Active,age_cat-Vintage-mean,age_cat-Vintage-std,age_cat-Vintage-cv,age_cat-Avg_Account_Balance-mean,age_cat-Avg_Account_Balance-std,age_cat-Avg_Account_Balance-cv,Gender-Vintage-mean,Gender-Vintage-std,Gender-Vintage-cv,Gender-Avg_Account_Balance-mean,Gender-Avg_Account_Balance-std,Gender-Avg_Account_Balance-cv,Region_Code-Vintage-mean,Region_Code-Vintage-std,Region_Code-Vintage-cv,Region_Code-Avg_Account_Balance-mean,Region_Code-Avg_Account_Balance-std,Region_Code-Avg_Account_Balance-cv,Occupation-Vintage-mean,Occupation-Vintage-std,Occupation-Vintage-cv,Occupation-Avg_Account_Balance-mean,Occupation-Avg_Account_Balance-std,Occupation-Avg_Account_Balance-cv,Channel_Code-Vintage-mean,Channel_Code-Vintage-std,Channel_Code-Vintage-cv,Channel_Code-Avg_Account_Balance-mean,Channel_Code-Avg_Account_Balance-std,Channel_Code-Avg_Account_Balance-cv,Credit_Product-Vintage-mean,Credit_Product-Vintage-std,Credit_Product-Vintage-cv,Credit_Product-Avg_Account_Balance-mean,Credit_Product-Avg_Account_Balance-std,Credit_Product-Avg_Account_Balance-cv,Is_Active-Vintage-mean,Is_Active-Vintage-std,Is_Active-Vintage-cv,Is_Active-Avg_Account_Balance-mean,Is_Active-Avg_Account_Balance-std,Is_Active-Avg_Account_Balance-cv,age_cat-Gender-Vintage-mean,age_cat-Gender-Vintage-std,age_cat-Gender-Vintage-cv,age_cat-Gender-Avg_Account_Balance-mean,age_cat-Gender-Avg_Account_Balance-std,age_cat-Gender-Avg_Account_Balance-cv,age_cat-Region_Code-Vintage-mean,age_cat-Region_Code-Vintage-std,age_cat-Region_Code-Vintage-cv,age_cat-Region_Code-Avg_Account_Balance-mean,age_cat-Region_Code-Avg_Account_Balance-std,age_cat-Region_Code-Avg_Account_Balance-cv,age_cat-Occupation-Vintage-mean,age_cat-Occupation-Vintage-std,age_cat-Occupation-Vintage-cv,age_cat-Occupation-Avg_Account_Balance-mean,age_cat-Occupation-Avg_Account_Balance-std,age_cat-Occupation-Avg_Account_Balance-cv,age_cat-Channel_Code-Vintage-mean,age_cat-Channel_Code-Vintage-std,age_cat-Channel_Code-Vintage-cv,age_cat-Channel_Code-Avg_Account_Balance-mean,age_cat-Channel_Code-Avg_Account_Balance-std,age_cat-Channel_Code-Avg_Account_Balance-cv,age_cat-Credit_Product-Vintage-mean,age_cat-Credit_Product-Vintage-std,age_cat-Credit_Product-Vintage-cv,age_cat-Credit_Product-Avg_Account_Balance-mean,age_cat-Credit_Product-Avg_Account_Balance-std,age_cat-Credit_Product-Avg_Account_Balance-cv,age_cat-Is_Active-Vintage-mean,age_cat-Is_Active-Vintage-std,age_cat-Is_Active-Vintage-cv,age_cat-Is_Active-Avg_Account_Balance-mean,age_cat-Is_Active-Avg_Account_Balance-std,age_cat-Is_Active-Avg_Account_Balance-cv,Gender-Region_Code-Vintage-mean,Gender-Region_Code-Vintage-std,Gender-Region_Code-Vintage-cv,Gender-Region_Code-Avg_Account_Balance-mean,Gender-Region_Code-Avg_Account_Balance-std,Gender-Region_Code-Avg_Account_Balance-cv,Gender-Occupation-Vintage-mean,Gender-Occupation-Vintage-std,Gender-Occupation-Vintage-cv,Gender-Occupation-Avg_Account_Balance-mean,Gender-Occupation-Avg_Account_Balance-std,Gender-Occupation-Avg_Account_Balance-cv,Gender-Channel_Code-Vintage-mean,Gender-Channel_Code-Vintage-std,Gender-Channel_Code-Vintage-cv,Gender-Channel_Code-Avg_Account_Balance-mean,Gender-Channel_Code-Avg_Account_Balance-std,Gender-Channel_Code-Avg_Account_Balance-cv,Gender-Credit_Product-Vintage-mean,Gender-Credit_Product-Vintage-std,Gender-Credit_Product-Vintage-cv,Gender-Credit_Product-Avg_Account_Balance-mean,Gender-Credit_Product-Avg_Account_Balance-std,Gender-

In [41]:
all_cat_cols = cat_cols + featured_cols
encoder = LabelEncoder()
for col in all_cat_cols:
    train[col] = encoder.fit_transform(train[col])
    test[col] = encoder.transform(test[col])
    valid[col] = encoder.transform(valid[col])

In [42]:
x_train = train.drop(['ID','Is_Lead'],axis=1)
y_train = train['Is_Lead']

x_valid = valid.drop(['ID','Is_Lead'],axis=1)
y_valid = valid['Is_Lead']

x_test = test.drop(['ID'],axis=1)

In [45]:
cat_indx = []
for idx, col in enumerate(x_train.columns):
    if col in all_cat_cols:
        cat_indx.append(idx)

In [ ]:
hist_params = {'max_iter':1000,
               'learning_rate' : 0.003,
               'max_depth' : 30,
               'early_stopping' : 'auto',
               'verbose':1,
               'max_bins' : 255,
               'random_state':636,
              
              }


hist = HistGradientBoostingClassifier(**hist_params)
hist.fit(x_train,y_train)
pred = hist.predict_proba(x_valid)[:, 1]
roc_score = roc_auc_score(y_valid, pred)
print(f"roc_auc_score: {roc_score}")

In [54]:
submit = pd.DataFrame()
submit['ID'] = test['ID']
submit['Is_Lead'] = hist.predict_proba(x_test)[:, 1]
submit.head(3)

,ID,Is_Lead
0,VBENBARO,0.059454
1,CCMEWNKY,0.822981
2,VK3KGA9M,0.062201


In [55]:
submit.to_csv("Hist_submit_v2.csv",index=False)